In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/movie_data/movie_data"))

# Any results you write to the current directory are saved as output.

['full_train.txt', 'full_test.txt']


In [5]:
import numpy as np
import pandas as pd
import os
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [6]:
reviews_train = []
for line in open('../input/movie_data/movie_data/full_train.txt', 'r'):
    
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open('../input/movie_data/movie_data/full_test.txt', 'r'):
    
    reviews_test.append(line.strip())
    
target = [1 if i < 12500 else 0 for i in range(25000)]

In [9]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [72]:
baseline_vectorizer = CountVectorizer(binary=True)
baseline_vectorizer.fit(reviews_train_clean)
print("Size of dictionary: ", len(baseline_vectorizer.get_feature_names()))
#print("Words in dictionary: ", baseline_vectorizer.get_feature_names())

Size of dictionary:  86374


In [73]:
X_baseline = baseline_vectorizer.transform(reviews_train_clean)
X_test_baseline = baseline_vectorizer.transform(reviews_test_clean)

In [74]:
X_train, X_val, y_train, y_val = train_test_split(X_baseline, target, train_size = 0.85)
score = []
c = [0.01, 0.05, 0.25, 0.5, 1]
for i in c:
    lr = LogisticRegression(C=i)
    lr.fit(X_train, y_train)
    score.append(accuracy_score(y_val, lr.predict(X_val)))
    print ("Accuracy for C=%s: %s" % (i, score[-1]))

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy for C=0.01: 0.8701333333333333
Accuracy for C=0.05: 0.8810666666666667
Accuracy for C=0.25: 0.8872
Accuracy for C=0.5: 0.8832
Accuracy for C=1: 0.8826666666666667


In [75]:
print("Value using for C: ", c[np.where(max(score) == score)[0][0]])
final_model = LogisticRegression(C=c[np.where(max(score) == score)[0][0]])
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test_baseline)))

Value using for C:  0.25


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Final Accuracy: 0.87528


**Remove stop words**

In [78]:
from nltk.corpus import stopwords

english_stop_words = stopwords.words('english')

def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

In [79]:
no_stop_words_train = remove_stop_words(reviews_train_clean)
no_stop_words_test = remove_stop_words(reviews_test_clean)

In [80]:
stopword_vectorizer = CountVectorizer(binary=True)
stopword_vectorizer.fit(no_stop_words_train)
print("Size of dictionary: ", len(stopword_vectorizer.get_feature_names()))
#print("Words in dictionary: ", stopword_vectorizer.get_feature_names())

Size of dictionary:  86357


In [81]:
X = stopword_vectorizer.transform(no_stop_words_train)
X_test = stopword_vectorizer.transform(no_stop_words_test)

In [84]:
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.85)
score = []
c = [0.01, 0.05, 0.25, 0.5, 1]
for i in c:
    lr = LogisticRegression(C=i)
    lr.fit(X_train, y_train)
    score.append(accuracy_score(y_val, lr.predict(X_val)))
    print ("Accuracy for C=%s: %s" % (i, score[-1]))

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy for C=0.01: 0.8802666666666666
Accuracy for C=0.05: 0.8872
Accuracy for C=0.25: 0.8829333333333333
Accuracy for C=0.5: 0.8824
Accuracy for C=1: 0.8805333333333333


In [86]:
print("Value using for C: ", c[np.where(max(score) == score)[0][0]])
final_model = LogisticRegression(C=c[np.where(max(score) == score)[0][0]])
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test)))

Value using for C:  0.05


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Final Accuracy: 0.87796


In [89]:
feature_to_coef = {
    word: coef for word, coef in zip(
        stopword_vectorizer.get_feature_names(), final_model.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)
    
print("\n\n")
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

('excellent', 0.8804602077347948)
('perfect', 0.7432126467211464)
('great', 0.6827123379158221)
('enjoyable', 0.6258012729095954)
('wonderful', 0.6087005468520406)
('amazing', 0.5903051684301432)
('superb', 0.5819249705470795)
('enjoyed', 0.5738280746218912)
('favorite', 0.5729119064279654)
('today', 0.5695702126588372)



('worst', -1.3815488254103057)
('waste', -1.1508659423383225)
('awful', -0.9480172016161376)
('boring', -0.8621169054367182)
('disappointment', -0.8452674232927643)
('poorly', -0.8108798474450853)
('bad', -0.7483155792783629)
('disappointing', -0.7437232895600603)
('dull', -0.7427827816056617)
('poor', -0.7347383332416065)


**Lemmatization**
https://www.datacamp.com/community/tutorials/stemming-lemmatization-python

In [90]:
from nltk.stem import WordNetLemmatizer

def get_lemmatized_text(corpus):
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

In [93]:
lemmatized_reviews_train = get_lemmatized_text(reviews_train_clean)
lemmatized_reviews_test = get_lemmatized_text(reviews_test_clean)

In [94]:
lemmatize_vectorizer = CountVectorizer(binary=True)
lemmatize_vectorizer.fit(lemmatized_reviews_train)
print("Size of dictionary: ", len(lemmatize_vectorizer.get_feature_names()))
#print("Words in dictionary: ", lemmatize_vectorizer.get_feature_names())

Size of dictionary:  79651


In [95]:
X = lemmatize_vectorizer.transform(lemmatized_reviews_train)
X_test = lemmatize_vectorizer.transform(lemmatized_reviews_test)

In [97]:
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.85)
score = []
c = [0.01, 0.05, 0.25, 0.5, 1]
for i in c:
    lr = LogisticRegression(C=i)
    lr.fit(X_train, y_train)
    score.append(accuracy_score(y_val, lr.predict(X_val)))
    print ("Accuracy for C=%s: %s" % (i, score[-1]))

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy for C=0.01: 0.8725333333333334
Accuracy for C=0.05: 0.8805333333333333
Accuracy for C=0.25: 0.8770666666666667
Accuracy for C=0.5: 0.8749333333333333
Accuracy for C=1: 0.8712


In [98]:
print("Value using for C: ", c[np.where(max(score) == score)[0][0]])
final_model = LogisticRegression(C=c[np.where(max(score) == score)[0][0]])
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test)))

Value using for C:  0.05


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Final Accuracy: 0.87812


In [99]:
feature_to_coef = {
    word: coef for word, coef in zip(
        lemmatize_vectorizer.get_feature_names(), final_model.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)
    
print("\n\n")
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

('excellent', 0.9084688438231432)
('perfect', 0.7578961370927583)
('great', 0.6625676769834377)
('amazing', 0.640343872065302)
('favorite', 0.6152024212167753)
('loved', 0.5902433107905792)
('enjoyable', 0.5854735864118813)
('wonderful', 0.5826678128955866)
('today', 0.572724060958529)
('superb', 0.5586111531811624)



('worst', -1.3157214030538866)
('waste', -1.0804376505618578)
('awful', -1.067378862497543)
('boring', -0.8524944764937644)
('poorly', -0.8219956324605929)
('disappointing', -0.7046667736899124)
('terrible', -0.6997226931439782)
('disappointment', -0.6960900492360738)
('poor', -0.6891741876904484)
('dull', -0.6851276557577579)


**Stemming**
https://www.datacamp.com/community/tutorials/stemming-lemmatization-python

In [104]:
from nltk.stem.porter import PorterStemmer

def get_stemmed_text(corpus):
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

In [105]:
stemmed_reviews_train = get_stemmed_text(reviews_train_clean)
stemmed_reviews_test = get_stemmed_text(reviews_test_clean)

In [106]:
stemm_vectorizer = CountVectorizer(binary=True)
stemm_vectorizer.fit(stemmed_reviews_train)
print("Size of dictionary: ", len(stemm_vectorizer.get_feature_names()))
#print("Words in dictionary: ", stemm_vectorizer.get_feature_names())

Size of dictionary:  66220


In [107]:
X = stemm_vectorizer.transform(stemmed_reviews_train)
X_test = stemm_vectorizer.transform(stemmed_reviews_test)

In [108]:
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.85)
score = []
c = [0.01, 0.05, 0.25, 0.5, 1]
for i in c:
    lr = LogisticRegression(C=i)
    lr.fit(X_train, y_train)
    score.append(accuracy_score(y_val, lr.predict(X_val)))
    print ("Accuracy for C=%s: %s" % (i, score[-1]))

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy for C=0.01: 0.8672
Accuracy for C=0.05: 0.8754666666666666
Accuracy for C=0.25: 0.8725333333333334
Accuracy for C=0.5: 0.8685333333333334
Accuracy for C=1: 0.8656


In [109]:
print("Value using for C: ", c[np.where(max(score) == score)[0][0]])
final_model = LogisticRegression(C=c[np.where(max(score) == score)[0][0]])
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test)))

Value using for C:  0.05


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Final Accuracy: 0.87648


In [110]:
feature_to_coef = {
    word: coef for word, coef in zip(
        stemm_vectorizer.get_feature_names(), final_model.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)
    
print("\n\n")
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

('excel', 0.9624390848254747)
('perfect', 0.7969511879296894)
('great', 0.6771041782527951)
('favorit', 0.6421194642722267)
('superb', 0.595193768080948)
('refresh', 0.5871849749417393)
('enjoy', 0.5657464323780484)
('today', 0.5636867721494785)
('highli', 0.5627563945597394)
('perfectli', 0.5597418079328637)



('worst', -1.3279368675211911)
('wast', -1.1230465034330532)
('aw', -0.974097904944865)
('dull', -0.8180963081852665)
('poorli', -0.8115788562939348)
('bore', -0.771575721698185)
('disappoint', -0.7681881659478987)
('poor', -0.753727826733088)
('wors', -0.7050334555348351)
('bad', -0.6753253325883405)


**Using n-grams**

In [111]:
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(reviews_train_clean)
print("Size of dictionary: ", len(ngram_vectorizer.get_feature_names()))
#print("Words in dictionary: ", ngram_vectorizer.get_feature_names())

Size of dictionary:  1532909


In [112]:
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

In [113]:
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.85)
score = []
c = [0.01, 0.05, 0.25, 0.5, 1]
for i in c:
    lr = LogisticRegression(C=i)
    lr.fit(X_train, y_train)
    score.append(accuracy_score(y_val, lr.predict(X_val)))
    print ("Accuracy for C=%s: %s" % (i, score[-1]))

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy for C=0.01: 0.8850666666666667
Accuracy for C=0.05: 0.8952
Accuracy for C=0.25: 0.8952
Accuracy for C=0.5: 0.8952
Accuracy for C=1: 0.8941333333333333


In [114]:
print("Value using for C: ", c[np.where(max(score) == score)[0][0]])
final_model = LogisticRegression(C=c[np.where(max(score) == score)[0][0]])
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test)))

Value using for C:  0.05


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Final Accuracy: 0.89412


In [115]:
feature_to_coef = {
    word: coef for word, coef in zip(
        ngram_vectorizer.get_feature_names(), final_model.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)
    
print("\n\n")
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

('excellent', 0.7171399759506399)
('great', 0.6532125292060386)
('perfect', 0.6385149399533783)
('wonderful', 0.5499892406590796)
('amazing', 0.49769641145502413)
('superb', 0.47393847397792827)
('today', 0.4320437625802879)
('enjoyable', 0.43101133809667325)
('enjoyed', 0.4139699928555225)
('loved', 0.4056242228452547)



('worst', -0.9192909223109731)
('awful', -0.8711853969273665)
('waste', -0.7360252459846508)
('bad', -0.7149357190939444)
('boring', -0.7084538590791247)
('poor', -0.6621730464393614)
('the worst', -0.6427910150433388)
('terrible', -0.6036168389487931)
('poorly', -0.586304979171294)
('dull', -0.5467282170513686)


**TF-IDF (term frequency-inverse data frequency)**
https://buhrmann.github.io/tfidf-analysis.html

In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', analyzer='word', ngram_range=(1, 2))
tfidf_vectorizer.fit(reviews_train_clean)
print("Size of dictionary: ", len(tfidf_vectorizer.get_feature_names()))
#print("Words in dictionary: ", tfidf_vectorizer.get_feature_names())

Size of dictionary:  1777206


In [117]:
X = tfidf_vectorizer.transform(reviews_train_clean)
X_test = tfidf_vectorizer.transform(reviews_test_clean)

In [118]:
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.85)
score = []
c = [0.01, 0.05, 0.25, 0.5, 1]
for i in c:
    lr = LogisticRegression(C=i)
    lr.fit(X_train, y_train)
    score.append(accuracy_score(y_val, lr.predict(X_val)))
    print ("Accuracy for C=%s: %s" % (i, score[-1]))

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy for C=0.01: 0.8245333333333333
Accuracy for C=0.05: 0.8344
Accuracy for C=0.25: 0.8581333333333333
Accuracy for C=0.5: 0.8749333333333333
Accuracy for C=1: 0.8824


In [119]:
print("Value using for C: ", c[np.where(max(score) == score)[0][0]])
final_model = LogisticRegression(C=c[np.where(max(score) == score)[0][0]])
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test)))

Value using for C:  1


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Final Accuracy: 0.86968


In [120]:
feature_to_coef = {
    word: coef for word, coef in zip(
        tfidf_vectorizer.get_feature_names(), final_model.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)
    
print("\n\n")
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

('great', 8.96394765251946)
('excellent', 6.194993740811503)
('best', 5.971685753815957)
('love', 5.169923358869189)
('wonderful', 4.9498828867269)
('perfect', 4.323149888836021)
('amazing', 4.251834241511996)
('favorite', 4.102620094326023)
('fun', 3.9116056430903616)
('loved', 3.793782384717356)



('bad', -9.965354117289802)
('worst', -8.94865946902623)
('awful', -6.2635527310533)
('waste', -5.881233218778964)
('boring', -5.690736298978455)
('poor', -5.349430678416386)
('worse', -4.80313370729508)
('terrible', -4.781690638038636)
('script', -4.4218678082570095)
('minutes', -4.348345354652207)


**Using linear Support Vector Machine**

In [121]:
from sklearn.svm import LinearSVC

lsvc_ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
lsvc_ngram_vectorizer.fit(reviews_train_clean)
print("Size of dictionary: ", len(lsvc_ngram_vectorizer.get_feature_names()))
#print("Words in dictionary: ", lsvc_ngram_vectorizer.get_feature_names())

Size of dictionary:  1532909


In [122]:
X = lsvc_ngram_vectorizer.transform(reviews_train_clean)
X_test = lsvc_ngram_vectorizer.transform(reviews_test_clean)

In [123]:
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.85)
score = []
c = [0.01, 0.05, 0.25, 0.5, 1]
for i in c:
    lsvc = LinearSVC(C=i)
    lsvc.fit(X_train, y_train)
    score.append(accuracy_score(y_val, lsvc.predict(X_val)))
    print ("Accuracy for C=%s: %s" % (i, score[-1]))

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Accuracy for C=0.01: 0.8904
Accuracy for C=0.05: 0.8912
Accuracy for C=0.25: 0.8898666666666667


/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy for C=0.5: 0.8901333333333333
Accuracy for C=1: 0.8898666666666667


In [124]:
print("Value using for C: ", c[np.where(max(score) == score)[0][0]])
final_model = LinearSVC(C=c[np.where(max(score) == score)[0][0]])
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test)))

Value using for C:  0.05
Final Accuracy: 0.89172


In [125]:
feature_to_coef = {
    word: coef for word, coef in zip(
        lsvc_ngram_vectorizer.get_feature_names(), final_model.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)
    
print("\n\n")
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

('excellent', 0.2840498601198245)
('perfect', 0.2043035423297832)
('great', 0.20069819503674716)
('enjoyable', 0.19622293623041642)
('superb', 0.18114288427951528)
('wonderful', 0.1786807689008129)
('today', 0.1664913876944546)
('amazing', 0.16244509859458678)
('brilliant', 0.15540396579820856)
('definitely worth', 0.152865317341006)



('worst', -0.32395233894228503)
('awful', -0.31880229263368215)
('boring', -0.29843532714545445)
('waste', -0.2574711423043509)
('terrible', -0.2573343762893931)
('bad', -0.2374586177099922)
('the worst', -0.23164610072638653)
('disappointing', -0.2257264154994067)
('dull', -0.22216393598426507)
('poor', -0.22156403436822783)


In [126]:
pos = ["I've seen this story before but my kids haven't. Boy with troubled past joins military, faces his past, falls in love and becomes a man. "
       "The mentor this time is played perfectly by Kevin Costner; An ordinary man with common everyday problems who lives an extraordinary "
       "conviction, to save lives. After losing his team he takes a teaching position training the next generation of heroes. The young troubled "
       "recruit is played by Kutcher. While his scenes with the local love interest are a tad stiff and don't generate enough heat to melt butter, "
       "he compliments Costner well. I never really understood Sela Ward as the neglected wife and felt she should of wanted Costner to quit out of "
       "concern for his safety as opposed to her selfish needs. But her presence on screen is a pleasure. The two unaccredited stars of this movie "
       "are the Coast Guard and the Sea. Both powerful forces which should not be taken for granted in real life or this movie. The movie has some "
       "slow spots and could have used the wasted 15 minutes to strengthen the character relationships. But it still works. The rescue scenes are "
       "intense and well filmed and edited to provide maximum impact. This movie earns the audience applause. And the applause of my two sons."]
print("Pos prediction: {}". format(lsvc.predict(lsvc_ngram_vectorizer.transform(pos))))

Pos prediction: [1]


In [135]:
neg = ["We do not want to send out an e-mail with a subject line that recipient(s) feel like avoiding.""
       "Think about your own reaction on e-mails with negative subject lines;"
       "do you even feel like opening them? Example, instead of a subject line that says ‘Delay in" 
       "ABC project schedule,’ the subject line can be ‘Changes in ABC project schedule.'"]
print("Neg prediction: {}". format(lsvc.predict(lsvc_ngram_vectorizer.transform(neg))))

Neg prediction: [0]
